In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import pandas as pd
import matplotlib as plt

import mpd

In [ ]:
# Will allow us to embed images in the notebook
%matplotlib inline
# change default plot size
plt.rcParams['figure.figsize'] = (15,10)

In [ ]:
mpd_all=mpd.load(spark, "onebig", 1)

# Build count vector 

Based on code frmo mpd-expore notebook

In [ ]:
pDF=mpd_all.select("pid", "tracks.track_uri")

from pyspark.ml.feature import CountVectorizer

cv = CountVectorizer(inputCol="track_uri", outputCol="features")
model = cv.fit(pDF)
result = model.transform(pDF)

In [ ]:
result.show()

In [ ]:
result.select("features").head(1)

from pyspark.mllib.linalg import Vectors

sv = Vectors.sparse(cv.getVocabSize())

In [ ]:
from pyspark.mllib.linalg import Vectors

Vectors.parse('(3, [0], [2])')

Note the above conversion of the string representation of a sparse vector to a sparse vector datatype shows how these two formats are displayed.

When using the show() method the string representation is returned.  When using the head on the feature we actually see a native sparse vector datatype in a row.  This shows countvectorizer is actually producing sparse vector types, it's only show() that turns it into a string.

Since the feture is actually a row with a sparse vector in it, may be able take the feature column as a sparse matrix already and perform the distance measures on it.

sv=Vectors.sparse(result.select("features").head(1))

In [ ]:
from pyspark.ml.feature import MinHashLSH
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col

In [ ]:
mh = MinHashLSH(inputCol="features", outputCol="hashes", numHashTables=5)

In [ ]:
result.count()

In [ ]:
model = mh.fit(result.limit(10))

In [ ]:
transformA = model.transform(result.limit(10))

In [ ]:
transformB = model.transform(result.limit(10))

In [ ]:
transformA.show()

In [ ]:
transformA.select("hashes").limit(1)

In [ ]:
model.approxSimilarityJoin(transformA, transformB, 0.6, distCol="JaccardDistance")

In [ ]:
jd=model.approxSimilarityJoin(transformA, transformB,10.0, distCol="JaccardDistance")\
    .select(col("datasetA.pid").alias("idA"),
            col("datasetB.pid").alias("idB"),
            col("JaccardDistance"))

In [ ]:
jd.show()

In [ ]:
train = result.rdd.sample(False, .01, 1)
train = train.toDF()

In [ ]:
train.show()

In [ ]:
model.approxNearestNeighbors(dfA, key, 2).show()

In [ ]:
from pyspark.sql.functions import col, expr, when

In [ ]:
result.show(1)

In [ ]:
transformA.select("hashes").show(5, truncate=False)

result.select("features").rdd.map(lambda x: x.numNonzeros()).take(5)

In [ ]:
result.dtypes

In [ ]:
result.explain

Able to access the column as a sparse vector type but this doesn't let me operate on it

In [ ]:
result.select("features").rdd.map(lambda x: type(x.features)).take(5)

Able to call the sparseVector methods on the x element.

To return this as a dataframe on which we can run summary stats, the key is to [cast the lambda result as a Row](https://spark.apache.org/docs/latest/sql-programming-guide.html#inferring-the-schema-using-reflection). The toDF() works on rows.

In [ ]:
from pyspark.sql import Row


sparselen=result.select("features").rdd.map(lambda x: Row(length=x.features.numNonzeros())).toDF()

In [ ]:
sparselen.printSchema()

The summary stats are promising and show we are close to the same averages as with the full play lists data set.

In [ ]:
sparselen.describe().show()

In [ ]:
origlen = result.select("track_uri").rdd.map(lambda x: Row(length=len(x[0]))).toDF()

In [ ]:
origlen.describe().show()